In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
from shelterCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHUstudent1"
shelter = AnimalShelter(username, password)

# Get data from MongoDB and prepare DataFrame
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)

# Filter configurations for each rescue type
filter_conditions = {
    'Water Rescue': {
        'breed': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
        'sex': 'Intact Female',
        'age_range': (26, 156)
    },
    'Mountain or Wilderness Rescue': {
        'breed': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
        'sex': 'Intact Male',
        'age_range': (26, 156)
    },
    'Disaster Rescue': {
        'breed': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
        'sex': 'Intact Male',
        'age_range': (20, 300)
    }
}

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    # Logo and link
    html.Div(
        html.A(
            html.Img(src='/assets/Grazioso_Salvare_Logo.png', style={'height': '200px', 'display': 'block', 'margin': 'auto'}),
            href="https://www.snhu.edu",
            target="_blank"
        ),
        style={'textAlign': 'center', 'padding': '20px'}
    ),
    html.Div("Created by Quinlin MacKenzie", style={'textAlign': 'center', 'fontSize': 18}),
    html.Hr(),
    
    # Filter buttons
    html.Div([
        html.Button('Water Rescue', id='water-button', n_clicks=0),
        html.Button('Mountain or Wilderness Rescue', id='mountain-button', n_clicks=0),
        html.Button('Disaster Rescue', id='disaster-button', n_clicks=0),
        html.Button('Reset', id='reset-button', n_clicks=0),
    ]),
    html.Br(),

    # Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_cell={'textAlign': 'left'},
        style_data_conditional=[{
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgba(0, 0, 0, 0.05)'
        }],
        filter_action='none',  
        sort_action='native',
        row_selectable='single'
    ),
    html.Br(),
    html.Hr(),

    html.Div([
        # Map Section
        html.Div(id='map-id', style={'display': 'inline-block', 'width': '50%', 'verticalAlign': 'top'}),
        
        # Pie Chart Section
        html.Div(id='pie-chart-id', style={'display': 'inline-block', 'width': '50%', 'verticalAlign': 'top'})
    ], style={'textAlign': 'center'}),
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update DataTable based on filter button clicks
@app.callback(
    Output('datatable-id', 'data'),
    [Input('water-button', 'n_clicks'),
     Input('mountain-button', 'n_clicks'),
     Input('disaster-button', 'n_clicks'),
     Input('reset-button', 'n_clicks')]
)
def update_table(water_clicks, mountain_clicks, disaster_clicks, reset_clicks):
    ctx = dash.callback_context
    if not ctx.triggered:
        return df.to_dict('records')  # Default: show all data

    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    filtered_df = df  # Start with the unfiltered DataFrame

    if button_id == 'water-button':
        filtered_df = df[
            (df['breed'].isin(filter_conditions['Water Rescue']['breed'])) &
            (df['sex_upon_outcome'] == filter_conditions['Water Rescue']['sex']) &
            (df['age_upon_outcome_in_weeks'].between(*filter_conditions['Water Rescue']['age_range']))
        ]
    elif button_id == 'mountain-button':
        filtered_df = df[
            (df['breed'].isin(filter_conditions['Mountain or Wilderness Rescue']['breed'])) &
            (df['sex_upon_outcome'] == filter_conditions['Mountain or Wilderness Rescue']['sex']) &
            (df['age_upon_outcome_in_weeks'].between(*filter_conditions['Mountain or Wilderness Rescue']['age_range']))
        ]
    elif button_id == 'disaster-button':
        filtered_df = df[
            (df['breed'].isin(filter_conditions['Disaster Rescue']['breed'])) &
            (df['sex_upon_outcome'] == filter_conditions['Disaster Rescue']['sex']) &
            (df['age_upon_outcome_in_weeks'].between(*filter_conditions['Disaster Rescue']['age_range']))
        ]

    return filtered_df.to_dict('records') if not filtered_df.empty else []

# Callback to update the map based on selected data in the DataTable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    return [
        dl.Map(style={'width': '100%', 'height': '400px'},
               center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']],
                      children=[
                          dl.Tooltip(dff.iloc[row]['breed']),
                          dl.Popup([
                              html.H1("Animal Name"),
                              html.P(dff.iloc[row]['name'] if dff.iloc[row]['name'] else "Unnamed")
                          ])
                      ])
        ])
    ]

# Additional chart (Pie chart for breed distribution)
@app.callback(
    Output('pie-chart-id', 'children'),
    [Input('datatable-id', 'data')]
)
def update_pie_chart(data):
    if not data:
        return html.Div("No data available for chart.")

    dff = pd.DataFrame(data)
    fig = px.pie(dff, names='breed', title="Breed Distribution")
    return dcc.Graph(figure=fig)

# Run the app
app.run_server(debug=True)


Dash app running on http://127.0.0.1:27695/
